In [1]:
import os
import boto3
import joblib
import mido
from miditoolkit import MidiFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from pathlib import Path
from tqdm.notebook import tqdm

import dotenv
dotenv.load_dotenv()

sns.set()

# Helper Functions

In [2]:
def get_pitch_range(midi):
    min_pitch = float('inf')
    max_pitch = float('-inf')

    for track in midi.instruments:
        for note in track.notes:
            min_pitch = min(min_pitch, note.pitch)
            max_pitch = max(max_pitch, note.pitch)

    return min_pitch, max_pitch

# Get MetaData

In [3]:
# read in the full metadataset
meta_df = pd.read_csv(f"../dataset/master_midi_meta_parse.csv")
meta_df

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
0,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001,NaN,"['Am', 'G', 'Dm', 'C', 'D']",4,string,classical,0
1,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002,NaN,"['Am', 'Dm', 'G', 'C', 'G7', 'F']",0,keyboard,classical,0
2,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003,NaN,"['F', 'E+', 'E', 'Am']",4,string,classical,0
3,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004,NaN,"['A#', 'Dm', 'G', 'C', 'F', 'Gm']",7,choir,classical,0
4,aminor,"[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005,NaN,"['Em', 'F', 'Am']",0,keyboard,classical,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72376,amajor,"[['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',...",unknown,12,169,soul,unknown,electric_guitar_jazz,unknown,4/4,70,79,val,e34a552e87b6bb8939bc35660d2d116c-5,unknown,"['G', 'A', 'Gm']",0,keyboard,soul,11
72377,gminor,"[['Gm', 'Gm', 'Gm', 'Gm', 'B', 'B', 'B', 'B', ...",unknown,3,100,oldie,unknown,soprano_sax,unknown,4/4,116,116,val,d0531034be492842b6d9bb32486f6a95-1,unknown,"['B', 'Gm']",5,brass,oldie,8
72378,emajor,"[['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',...",unknown,11,180,rock,unknown,bright_acoustic_piano,unknown,2/4,95,95,val,72b9d074542398fc021f636eb17da32b-5,unknown,['B'],0,keyboard,rock,10
72379,c#major,"[['G#', 'G#', 'G#', 'G#', 'C#', 'C#', 'C#', 'C...",unknown,11,240,metal,unknown,acoustic_grand_piano,unknown,4/4,95,95,val,0bc159de1686720e1c0282b57a3af7e5-4,unknown,"['C#', 'G#', 'E']",0,keyboard,metal,7


In [4]:
# isolate commu only code
commu_meta = meta_df[meta_df["id"].str.startswith("commu")].reset_index(drop=True)
meta_midi = meta_df.drop(commu_meta.index).reset_index(drop=True)
commu_meta, meta_midi

(      audio_key                                 chord_progressions  \
 0        aminor  [['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...   
 1        cmajor  [['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...   
 2        aminor  [['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...   
 3        cmajor  [['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...   
 4        aminor  [['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...   
 ...         ...                                                ...   
 11139    cmajor  [['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7',...   
 11140    cmajor  [['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...   
 11141    aminor  [['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...   
 11142    aminor  [['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...   
 11143    aminor  [['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...   
 
       pitch_range  num_measures  bpm      genre     track_role  \
 0             mid             8  120  cinematic    main_melody   
 1         m

In [5]:
commu_meta

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
0,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001,NaN,"['Am', 'G', 'Dm', 'C', 'D']",4,string,classical,0
1,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002,NaN,"['Am', 'Dm', 'G', 'C', 'G7', 'F']",0,keyboard,classical,0
2,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003,NaN,"['F', 'E+', 'E', 'Am']",4,string,classical,0
3,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004,NaN,"['A#', 'Dm', 'G', 'C', 'F', 'Gm']",7,choir,classical,0
4,aminor,"[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005,NaN,"['Em', 'F', 'Am']",0,keyboard,classical,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,cmajor,"[['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7',...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140,NaN,"['Gsus4', 'G', 'Am7', 'C', 'Em7', 'G7', 'Fmaj7...",4,string,classical,0
11140,cmajor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141,NaN,"['G', 'Cmaj7', 'Fmaj7', 'Am']",3,guitar,classical,0
11141,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142,NaN,"['Am7', 'F', 'Am']",4,string,classical,0
11142,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143,NaN,"['Am7', 'Am']",4,string,classical,0


In [6]:
## TODO: could make everything in metamidi standard for sample rhythm
commu_meta.groupby(['track_role']).count()#.count().sort_values(['id'])

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
track_role,,,,,,,,,,,,,,,,,,,
accompaniment,2510,2510,2510,2510,2510,2510,2510,2510,2510,2510,2510,2510,2510,0,2510,2510,2510,2510,2510
bass,503,503,503,503,503,503,503,503,503,503,503,503,503,0,503,503,503,503,503
main_melody,3048,3048,3048,3048,3048,3048,3048,3048,3048,3048,3048,3048,3048,0,3048,3048,3048,3048,3048
pad,1720,1720,1720,1720,1720,1720,1720,1720,1720,1720,1720,1720,1720,0,1720,1720,1720,1720,1720
riff,1051,1051,1051,1051,1051,1051,1051,1051,1051,1051,1051,1051,1051,0,1051,1051,1051,1051,1051
sub_melody,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,0,2312,2312,2312,2312,2312


In [7]:
# for meta_midi filter out all instances where min_velocity == max_velocity
meta_midi = meta_midi[meta_midi["min_velocity"] != meta_midi["max_velocity"]].reset_index(drop=True)

In [8]:
# determine min and max BPM for commu
# TODO: set max BPM to 200
min_bpm = commu_meta["bpm"].min()
max_bpm = commu_meta["bpm"].max()
min_bpm, max_bpm, meta_midi['bpm'].min(), meta_midi['bpm'].max()

(35, 155, 30, 300)

In [9]:
# determine min and max BPM for commu
min_vel = commu_meta["min_velocity"].min()
max_vel = commu_meta["max_velocity"].max()
min_vel, max_vel, meta_midi['min_velocity'].min(), meta_midi['max_velocity'].max()

(2, 127, 1, 127)

In [10]:
# determine commu min/max number of measures
min_measures = commu_meta["num_measures"].min()
max_measures = commu_meta["num_measures"].max()
min_measures, max_measures, meta_midi['num_measures'].min(), meta_midi['num_measures'].max()

(4, 17, 1, 191)

In [11]:
# filter meta midi files such that it consistent with commu
## 1. filtering out bpm
meta_midi = meta_midi[meta_midi['bpm'].between(30, 200)].reset_index(drop=True)

## 2. filtering out velocity
meta_midi = meta_midi[meta_midi['min_velocity'].between(min_vel, max_vel)].reset_index(drop=True)

## 3. filtering out time signature
commu_time_sig = ["4/4", "3/4", "6/8", "12/8"]
meta_midi = meta_midi[meta_midi['time_signature'].isin(commu_time_sig)].reset_index(drop=True)

## 4. filtering out commu num_measures ranges
meta_midi = meta_midi[meta_midi['num_measures'].between(min_measures, 23)].reset_index(drop=True)


In [12]:
meta_midi

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
0,dmajor,"[['E', 'E', 'E', 'E', 'D', 'D', 'D', 'D', 'G',...",unknown,11,166,pop,unknown,lead_sawtooth,unknown,4/4,72,127,train,001344339e5b6a6bf1bc16d70b7f91a2-11,unknown,"['B', 'G', 'D', 'A', 'E']",1,accordian,pop,9
1,eminor,"[['F#', 'F#', 'F#', 'F#', 'E', 'E', 'E', 'E', ...",unknown,16,122,pop,unknown,lead_sawtooth,unknown,4/4,55,127,train,001344339e5b6a6bf1bc16d70b7f91a2-12,unknown,"['B', 'F#', 'G', 'C#', 'D', 'Gm', 'A', 'E']",1,accordian,pop,9
2,gmajor,"[['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'E',...",unknown,12,161,pop,unknown,overdriven_guitar,unknown,4/4,53,127,train,001344339e5b6a6bf1bc16d70b7f91a2-14,unknown,"['B', 'F#', 'G', 'C', 'D', 'A', 'E']",3,guitar,pop,9
3,gmajor,"[['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'E',...",unknown,12,155,pop,unknown,electric_guitar_muted,unknown,4/4,28,127,train,001344339e5b6a6bf1bc16d70b7f91a2-15,unknown,"['B', 'G', 'C', 'C#', 'D', 'A', 'E']",3,guitar,pop,9
4,aminor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'F',...",unknown,11,145,pop,unknown,melodic_tom,unknown,4/4,89,127,train,001344339e5b6a6bf1bc16d70b7f91a2-16,unknown,"['B', 'Am', 'G', 'C', 'F', 'D', 'A', 'E']",3,guitar,pop,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22575,emajor,"[['C#m', 'C#m', 'C#m', 'C#m', 'C#m', 'C#m', 'C...",unknown,9,174,rock,unknown,fx_scifi,unknown,4/4,59,68,val,47c1439cd723b457c9009516ac737279-2,unknown,"['B', 'G#m', 'C#m']",7,choir,rock,10
22576,gmajor,"[['B', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'G',...",unknown,10,188,electronic,unknown,acoustic_grand_piano,unknown,4/4,40,110,val,ac86c10bac321f338638cc5d49b72a43-1,unknown,"['B', 'F#', 'G', 'C', 'D', 'Em', 'A']",0,keyboard,electronic,3
22577,gmajor,"[['G', 'G', 'G', 'G', 'C', 'C', 'C', 'C', 'Am'...",unknown,16,140,country,unknown,string_ensemble,unknown,4/4,48,66,val,9290b0d249e42d455ad6ecda4fe0783a-15,unknown,"['D', 'G', 'C', 'Am']",4,string,country,1
22578,amajor,"[['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',...",unknown,12,169,soul,unknown,electric_guitar_jazz,unknown,4/4,70,79,val,e34a552e87b6bb8939bc35660d2d116c-5,unknown,"['G', 'A', 'Gm']",0,keyboard,soul,11


In [13]:
## use FAD score to assign track roles
commu_meta.groupby(['track_role']).count()

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
track_role,,,,,,,,,,,,,,,,,,,
accompaniment,2510,2510,2510,2510,2510,2510,2510,2510,2510,2510,2510,2510,2510,0,2510,2510,2510,2510,2510
bass,503,503,503,503,503,503,503,503,503,503,503,503,503,0,503,503,503,503,503
main_melody,3048,3048,3048,3048,3048,3048,3048,3048,3048,3048,3048,3048,3048,0,3048,3048,3048,3048,3048
pad,1720,1720,1720,1720,1720,1720,1720,1720,1720,1720,1720,1720,1720,0,1720,1720,1720,1720,1720
riff,1051,1051,1051,1051,1051,1051,1051,1051,1051,1051,1051,1051,1051,0,1051,1051,1051,1051,1051
sub_melody,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,2312,0,2312,2312,2312,2312,2312


In [14]:
## TODO: filtered on commu NUM Measures
commu_meta.groupby(['num_measures']).count()

,audio_key,chord_progressions,pitch_range,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
num_measures,,,,,,,,,,,,,,,,,,,
4,2710,2710,2710,2710,2710,2710,2710,2710,2710,2710,2710,2710,2710,0,2710,2710,2710,2710,2710
5,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2
8,7233,7233,7233,7233,7233,7233,7233,7233,7233,7233,7233,7233,7233,0,7233,7233,7233,7233,7233
9,80,80,80,80,80,80,80,80,80,80,80,80,80,0,80,80,80,80,80
16,1042,1042,1042,1042,1042,1042,1042,1042,1042,1042,1042,1042,1042,0,1042,1042,1042,1042,1042
17,77,77,77,77,77,77,77,77,77,77,77,77,77,0,77,77,77,77,77


In [15]:
meta_midi.groupby(['num_measures']).count()

,audio_key,chord_progressions,pitch_range,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
num_measures,,,,,,,,,,,,,,,,,,,
4,596,596,596,596,596,596,596,596,596,596,596,596,596,596,596,596,596,596,596
5,445,445,445,445,445,445,445,445,445,445,445,445,445,445,445,445,445,445,445
6,598,598,598,598,598,598,598,598,598,598,598,598,598,598,598,598,598,598,598
7,863,863,863,863,863,863,863,863,863,863,863,863,863,863,863,863,863,863,863
8,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075,1075
9,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203,1203
10,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276
11,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764,2764
12,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368,2368


In [120]:
measures = meta_midi.groupby(['num_measures']).count()
measures[measures.index > 17]['id'].sum() / measures['id'].sum()

0.1538086802480071

In [121]:
meta_midi.to_csv("../dataset/filtered_meta_midi_to_be_in_commu_range.csv", index=False)

In [122]:
meta_midi[meta_midi['genre'] == 'rock'].groupby(['split_data']).count()

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
split_data,,,,,,,,,,,,,,,,,,,
train,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803,5803
val,614,614,614,614,614,614,614,614,614,614,614,614,614,614,614,614,614,614,614


In [123]:
commu_meta[commu_meta['genre'] == 'newage'].groupby(['split_data']).count()

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
split_data,,,,,,,,,,,,,,,,,,,
train,2928,2928,2928,2928,2928,2928,2928,2928,2928,2928,2928,2928,2928,0,2928,2928,2928,2928,2928
val,192,192,192,192,192,192,192,192,192,192,192,192,192,0,192,192,192,192,192


In [124]:
# filter out data and append
# commu_meta = commu_meta[commu_meta['genre'] != 'newage']
# meta_midi = meta_midi[meta_midi['genre'] == 'rock']

final_df = pd.concat([commu_meta, meta_midi],axis=0).sample(frac=1).reset_index(drop=True)
final_df_train = final_df[final_df['split_data'] == 'train'].sample(frac=1).reset_index(drop=True)
final_df_test = final_df[final_df['split_data'] == 'val'].sample(frac=1).reset_index(drop=True)
final_df = pd.concat([final_df_train, final_df_test],axis=0).reset_index(drop=True)
final_df.to_csv("../dataset/meta_filtered_commu_full_inst_genre.csv", index=False)

In [125]:
final_df

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,track_roll,unique_chord_n_note,inst_mapping,updated_inst,updated_genre,genre_mapping
0,aminor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'B',...",unknown,7,36,rock,unknown,synthbrass,unknown,4/4,97,101,train,ecbea432cc241d9ce6d538e0151de1b7-6,unknown,"['B', 'G', 'C', 'F', 'Bm', 'Em', 'A']",5,brass,rock,10
1,cmajor,"[['G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G',...",unknown,8,180,pop,unknown,timpani,unknown,4/4,72,127,train,5daf3586f16c43682a80bcaa5c5a0da0-7,unknown,"['G', 'C', 'D', 'A', 'E']",6,percussion,pop,9
2,f#minor,"[['C#', 'C#', 'C#', 'C#', 'C#', 'C#', 'C#', 'C...",unknown,16,160,metal,unknown,overdriven_guitar,unknown,4/4,70,95,train,20edb760f18a93664d53dbfc66adc0e9-1,unknown,"['B', 'G', 'G#', 'C#', 'A']",3,guitar,metal,7
3,cmajor,"[['C', 'C', 'C', 'C', 'G', 'G', 'G', 'G', 'Am7...",mid_low,4,100,newage,accompaniment,acoustic_piano,standard,4/4,49,57,train,commu02701,NaN,"['G', 'Am7', 'C', 'C7', 'Em7', 'F', 'Dm7', 'Gm']",0,keyboard,classical,0
4,emajor,"[['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',...",unknown,7,153,oldie,unknown,percussive_organ,unknown,4/4,82,126,train,774575eebc32574da278f10b6f54acb2-5,unknown,"['B', 'F', 'E']",0,keyboard,oldie,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33719,dmajor,"[['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'D',...",unknown,4,141,folk_world_country,unknown,shanai,unknown,4/4,105,115,val,1da60045575d51727acd88155f663755-5,unknown,"['D', 'A']",5,brass,folk,4
33720,amajor,"[['Cm', 'Cm', 'Cm', 'Cm', 'G#', 'G#', 'G#', 'G...",unknown,7,191,pop,unknown,fx_crystal,unknown,4/4,88,122,val,f31c766ba03601e5f07eeeb4b0018825-8,unknown,"['D', 'Cm', 'G#', 'E']",5,brass,pop,9
33721,fminor,"[['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',...",unknown,18,90,reggae,unknown,acoustic_bass,unknown,4/4,63,111,val,697c38add4bcb7c403a388033db3c450-1,unknown,"['B', 'G#', 'C', 'F', 'E']",3,guitar,pop,9
33722,aminor,"[['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E',...",unknown,18,120,rock,unknown,electric_guitar_muted,unknown,4/4,95,127,val,710bd2eb118ed2bd632a701f6cba6546-1,unknown,"['A', 'C', 'E']",3,guitar,rock,10


In [126]:
final_df[final_df['genre_mapping'] == 0].groupby(['genre_mapping', 'genre']).count()

audio_key  chord_progressions  pitch_range  \
genre_mapping genre                                                   
0             cinematic       8024                8024         8024   
              classical        976                 976          976   
              gothic            13                  13           13   
              latin            280                 280          280   
              newage          3155                3155         3155   

                         num_measures   bpm  track_role  inst  sample_rhythm  \
genre_mapping genre                                                            
0             cinematic          8024  8024        8024  8024           8024   
              classical           976   976         976   976            976   
              gothic               13    13          13    13             13   
              latin               280   280         280   280            280   
              newage             3155  3155        3155  3155           3155   

                         time_signature  min_velocity  max_velocity  \
genre_mapping genre                                                   
0             cinematic            8024          8024          8024   
              classical             976           976           976   
              gothic                 13            13            13   
              latin                 280           280           280   
              newage               3155          3155          3155   

                         split_data    id  track_roll  unique_chord_n_note  \
genre_mapping genre                                                          
0             cinematic        8024  8024           0                 8024   
              classical         976   976         976                  976   
              gothic             13    13          13                   13   
              latin             280   280         280                  280   
              newage           3155  3155          35                 3155   

                         inst_mapping  updated_inst  updated_genre  
genre_mapping genre                                                 
0             cinematic          8024          8024           8024  
              classical           976           976            976  
              gothic               13            13             13  
              latin               280           280            280  
              newage             3155          3155           3155

In [127]:
# # Filtered out meta_midi only
# final_df_train = meta_midi[meta_midi['split_data'] == 'train'].sample(frac=1).reset_index(drop=True)
# final_df_test = meta_midi[meta_midi['split_data'] == 'val'].sample(frac=1).reset_index(drop=True)
# final_df = pd.concat([final_df_train, final_df_test],axis=0).reset_index(drop=True)

In [128]:
# import shutil
# # move the data to the respective folder
# train_raw = f"../dataset/complete_midi/train/raw"
# val_raw = f"../dataset/complete_midi/val/raw"

# train_dir = f"../dataset/meta_midi_only_filtered/train/raw"
# val_dir = f"../dataset/meta_midi_only_filtered/val/raw"

# # create the folder if not exist
# Path(train_dir).mkdir(parents=True, exist_ok=True)
# Path(val_dir).mkdir(parents=True, exist_ok=True)

# # copy the data using the split_data column
# for i, row in tqdm(final_df.iterrows()):
#     try:
#         # copy the file and replace underscore with hyphen
#         # also replace the id with same structure
#         if row['split_data'] == 'train':
#             shutil.copy(f"{train_raw}/{row['id']}.mid", f"{train_dir}/{row['id'].replace('_', '-')}.mid")
#         elif row['split_data'] == 'val':
#             shutil.copy(f"{val_raw}/{row['id']}.mid", f"{val_dir}/{row['id'].replace('_', '-')}.mid")
#     except Exception as e:
#         print(e)
#         pass


In [129]:
# final_df.to_csv("../dataset/meta_midi_only_filtered.csv", index=False)

In [130]:
# import json
# genre_map = final_df[['genre', 'genre_mapping']].groupby(['genre']).max().to_dict()['genre_mapping']
# with open(f"../dataset/genre_map_v2.json", 'w') as f:
#     f.write(json.dumps(genre_map, indent=4))

# Increase Batch Size
# Filling Unknown
### 1. standard fill for sample rythm
### 2. model to classify pitch ranges
### 3. randomly assign to reach commu distribution

# Pitch Ranges Analysis

In [131]:
# determine the pitch range of the midi files
commu_pitch_ranges = []
for i, row in tqdm(commu_meta.iterrows()):
    midi_file_path = f"../dataset/complete_midi/{row['split_data']}/raw/{row['id']}.mid"

    try:
        # extract the pitch range
        midi = MidiFile(midi_file_path)
        min_pitch, max_pitch = get_pitch_range(midi)

        
        if min_pitch == float('inf') or max_pitch == float('-inf'):
            print("No pitch information found in the MIDI file.")
        else:
            # print(f"Pitch Range: {min_pitch} to {max_pitch}")
            commu_pitch_ranges.append([row['id'], row['pitch_range'], min_pitch, max_pitch])

    except Exception as e:
        print(e)
        pass



0it [00:00, ?it/s]

In [141]:
# compute mean 
commu_pitch_ranges_df  = pd.DataFrame(commu_pitch_ranges, columns=["id", "pitch_range", "min_pitch", "max_pitch"])
commu_pitch_ranges_df

,id,pitch_range,min_pitch,max_pitch
0,commu00001,mid,57,81
1,commu00002,mid_low,41,67
2,commu00003,mid_high,71,89
3,commu00004,mid,62,76
4,commu00005,mid_low,40,60
...,...,...,...,...
11139,commu11140,mid_high,60,91
11140,commu11141,mid_low,53,67
11141,commu11142,very_low,33,33
11142,commu11143,low,40,48


In [142]:
quantiles = [0.25, 0.75]  # You can adjust these as needed

# Group by and aggregate other statistics
summary_stats = commu_pitch_ranges_df.groupby("pitch_range").agg({
    "min_pitch": ['min', 'max', 'mean', 'median', lambda x: x.quantile(quantiles).tolist()[0], lambda x: x.quantile(quantiles).tolist()[1]],
    "max_pitch": ['min', 'max', 'mean', 'median', lambda x: x.quantile(quantiles).tolist()[0], lambda x: x.quantile(quantiles).tolist()[1]]
})

# rename the columns by merging the multiindex columns together
summary_stats = summary_stats.rename(columns={'<lambda_0>': 'q25',
                                              '<lambda_1>': 'q75'})
summary_stats.columns = summary_stats.columns.map('_'.join)
summary_stats = summary_stats.reset_index()
summary_stats

,pitch_range,min_pitch_min,min_pitch_max,min_pitch_mean,min_pitch_median,min_pitch_q25,min_pitch_q75,max_pitch_min,max_pitch_max,max_pitch_mean,max_pitch_median,max_pitch_q25,max_pitch_q75
0,high,62,93,79.256516,79.0,76.00,83.00,84,108,94.775034,95.0,91.0,98.0
1,low,22,46,36.157895,36.0,33.00,40.00,33,69,47.969925,48.0,45.0,52.0
2,mid,26,71,54.717620,55.0,50.00,60.00,55,102,74.820012,76.0,72.0,79.0
3,mid_high,28,83,68.294976,69.0,65.00,72.00,64,103,86.059157,86.0,83.0,90.0
4,mid_low,5,59,42.547331,41.0,39.00,47.00,45,101,65.166904,65.0,62.0,69.0
5,very_high,86,95,90.500000,90.5,88.25,92.75,100,108,104.000000,104.0,102.0,106.0
6,very_low,21,33,28.486486,29.0,26.00,31.00,24,45,35.797297,36.0,33.0,37.0


In [143]:
commu_pitch_ranges_df = commu_pitch_ranges_df.merge(summary_stats, on="pitch_range")
commu_pitch_ranges_df

,id,pitch_range,min_pitch,max_pitch,min_pitch_min,min_pitch_max,min_pitch_mean,min_pitch_median,min_pitch_q25,min_pitch_q75,max_pitch_min,max_pitch_max,max_pitch_mean,max_pitch_median,max_pitch_q25,max_pitch_q75
0,commu00001,mid,57,81,26,71,54.717620,55.0,50.00,60.00,55,102,74.820012,76.0,72.0,79.0
1,commu00004,mid,62,76,26,71,54.717620,55.0,50.00,60.00,55,102,74.820012,76.0,72.0,79.0
2,commu00008,mid,67,69,26,71,54.717620,55.0,50.00,60.00,55,102,74.820012,76.0,72.0,79.0
3,commu00013,mid,65,76,26,71,54.717620,55.0,50.00,60.00,55,102,74.820012,76.0,72.0,79.0
4,commu00017,mid,53,77,26,71,54.717620,55.0,50.00,60.00,55,102,74.820012,76.0,72.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,commu11134,very_low,26,36,21,33,28.486486,29.0,26.00,31.00,24,45,35.797297,36.0,33.0,37.0
11140,commu11142,very_low,33,33,21,33,28.486486,29.0,26.00,31.00,24,45,35.797297,36.0,33.0,37.0
11141,commu11144,very_low,33,33,21,33,28.486486,29.0,26.00,31.00,24,45,35.797297,36.0,33.0,37.0
11142,commu01427,very_high,86,108,86,95,90.500000,90.5,88.25,92.75,100,108,104.000000,104.0,102.0,106.0
